In [15]:
import sys
sys.path.append('../')
from pathlib import Path
import mdtraj as md
import torch
import numpy as np

In [5]:
from mdn_propagator.propagator import Propagator

In [6]:
pdb = '../../CMSC-35450/data_mdshare/alanine-dipeptide-nowater.pdb'
files = sorted([str(i) for i in Path('../../CMSC-35450/data_mdshare').glob('alanine-dipeptide-*-250ns-nowater.xtc')])

In [9]:
trjs = [md.load(f, top=pdb) for f in files]
trj = md.load(files, top=pdb)
trj, len(trjs)

(<mdtraj.Trajectory with 750000 frames, 22 atoms, 3 residues, and unitcells at 0x7f430e588780>,
 3)

In [16]:
phi_psi_data = [np.concatenate((md.compute_phi(t)[1],md.compute_psi(t)[1]), -1) for t in trjs]
len(phi_psi_data), phi_psi_data[0].shape

(3, (250000, 2))

In [19]:
phi_psi_data = [torch.tensor(p).float() for p in phi_psi_data]

In [20]:
model = Propagator(dim = 2)

In [ ]:
model.fit(phi_psi_data, lag = 1)

/project/andrewferguson/Kirill/class_project_env/lib/python3.7/site-packages/lightning_lite/plugins/environments/slurm.py:170: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /project/andrewferguson/Kirill/class_project_env/lib ...
  category=PossibleUserWarning,
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name | Type                  | Params
-----------------------------------------------
0 | mdn  | MixtureDensityNetwork | 33.0 K
-----------------------------------------------
33.0 K    Trainable params
0         Non-trainable params
33.0 K    Total params
0.132     Total estimated model params size (MB)
/project/andrewferguson/Kirill/class_project_env/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connect

Training: 0it [00:00, ?it/s]

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [24]:
model.eval()

Propagator(
  (mdn): MixtureDensityNetwork(
    (network): MLP(
      (mlp): Sequential(
        (0): Linear(in_features=2, out_features=128, bias=True)
        (1): SiLU()
        (2): Linear(in_features=128, out_features=128, bias=True)
        (3): SiLU()
        (4): Linear(in_features=128, out_features=125, bias=True)
      )
    )
  )
)

In [ ]:
model.

In [ ]:
with torch.no_grad():
    x0 = torch.tensor(data_scaled[0][0]).float()[None]
    xt = torch.tensor(data_scaled[0][data.dataset.lag]).float()[None]
    x = torch.cat((x0,xt),-1).to(model.device)
    xs = list()
    for _ in tqdm(range(int(2E5))):
        x_next = model.mdn.sample(x).clip(0, 1)
        xs.append(x_next)
        x = torch.cat((x[:, x0.shape[-1]:], x_next), -1)
xs = torch.cat(xs)
pred_phi_psi = scaler.inverse_transform(xs.cpu().numpy())